In [36]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from tensorflow.keras import backend as K
import numpy as np

In [31]:
dataset_filename = 'full_dataset_xyz.csv'

In [32]:
all_df = pd.read_csv(dataset_filename)

all_df.head(3)

,timestamp,left_x,left_y,left_z,right_x,right_y,right_z,Activity
0,2024-07-19 20:00:07,-241.647059,-100.941176,-79.843137,-235.172414,30.482759,52.137931,none
1,2024-07-19 20:00:08,-245.200000,-86.320000,-76.240000,-252.880000,18.720000,43.040000,none
2,2024-07-19 20:00:09,-248.235294,-80.470588,-71.450980,-244.240000,16.240000,76.560000,none


In [33]:
# Convert the 'timestamp' column to datetime
day_df = all_df.copy()
day_df['timestamp'] = pd.to_datetime(day_df['timestamp'])

# Filter data only for the date 2024-07-27
date_of_interest = "2024-07-27"
filtered_data = day_df[day_df['timestamp'].dt.date == pd.to_datetime(date_of_interest).date()]

print(filtered_data.head(3))

                 timestamp      left_x     left_y      left_z    right_x  \
619193 2024-07-27 00:00:00 -207.921569  59.921569 -151.843137  39.183673   
619194 2024-07-27 00:00:01 -205.760000  60.000000 -152.800000  37.632653   
619195 2024-07-27 00:00:02 -204.160000  59.920000 -155.520000  36.244898   

          right_y     right_z Activity  
619193 -21.306122 -244.081633     none  
619194 -20.979592 -244.326531     none  
619195 -22.040816 -244.408163     none  


In [34]:
raw_data = filtered_data[['left_x','left_y','left_z','right_x','right_y','right_z']]

X = raw_data.to_numpy()

In [35]:
X.shape

(28802, 6)

In [25]:
focal_loss = SigmoidFocalCrossEntropy(alpha=0.25, gamma=2.0)

def fbeta(y_true, y_pred, beta=1):
    y_pred = K.round(y_pred)  # Round predictions to 0 or 1
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)
    
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    beta_squared = beta ** 2
    fbeta_score = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return K.mean(fbeta_score)

# Load your H5 model
model = load_model('best_model.h5', custom_objects={'fbeta': fbeta, 'SigmoidFocalCrossEntropy': SigmoidFocalCrossEntropy})

In [ ]:
def sliding_window_predict(original_matrix, model, chunk_size=300):
    n_rows = original_matrix.shape[0]
    predictions_list = []
    # Loop through the matrix in chunks of `chunk_size`
    for start in range(0, n_rows-chunk_size+1, chunk_size):
        end = start + chunk_size
        if end > n_rows:
            end = n_rows
        # Extract the chunk of the matrix
        chunk = original_matrix[start:end]

        # Predict using the model
        predictions = model.predict(np.expand_dims(chunk, axis=0))[0]

        # Append the predictions to the list
        predictions_list.append(predictions)

    # Combine all predictions back together into a single matrix
    final_predictions = np.vstack(predictions_list)

    return final_predictions


final_predictions = sliding_window_predict(X, model)

0
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 20ms/step
2
1/1 [==============================] - 0s 22ms/step
3
1/1 [==============================] - 0s 21ms/step
4
1/1 [==============================] - 0s 21ms/step
5
1/1 [==============================] - 0s 22ms/step
6
1/1 [==============================] - 0s 21ms/step
7
1/1 [==============================] - 0s 20ms/step
8
1/1 [==============================] - 0s 19ms/step
9
1/1 [==============================] - 0s 20ms/step
10
1/1 [==============================] - 0s 21ms/step
11
1/1 [==============================] - 0s 20ms/step
12
1/1 [==============================] - 0s 19ms/step
13
1/1 [==============================] - 0s 21ms/step
14
1/1 [==============================] - 0s 23ms/step
15
1/1 [==============================] - 0s 22ms/step
16
1/1 [==============================] - 0s 20ms/step
17
1/1 [==============================] - 0s 19ms/step
18
1/1 [============

In [43]:
numerical_predictions = np.argmax(final_predictions, axis=-1)

numerical_predictions.shape

(28800,)

In [49]:
activity_mapping = {
    0: 'S',
    1: 'A',
    2: 'P',
    3: 'T',
    4: 'M',
    5: 'kiné',
    6: 'none'
}